In [3]:
#beautifulsoup to work with the pages
from bs4 import BeautifulSoup
#used to access the pages without having problem to screaping
import requests
#work with dataframes and better manipulation of data
import pandas as pd
#used to create folders and files
import os
#used to create the .tsv files
import csv
#a particular string matches a given regular expression 
import re
#use to detect the language
from langdetect import detect

In [36]:
#1.1
#define an array to load the urls
url = []
#this for is to load the pages
for i in range(1, 301):
    #request access to the page number i
    page = requests.get("https://www.goodreads.com/list/show/1.Best_Books_Ever?page="+ str(i))
    #read properly the html page to scrape data
    soup = BeautifulSoup(page.content, features='lxml')
    #find the url of the given book
    find_a = soup.find_all('a',class_="bookTitle", itemprop = "url")
    #in each page we have 100 books, this inside for is to fill the array with the url of each book on the page
    for j in range(0,100):
        #to add the book url on the array
        url.append("https://www.goodreads.com"+ find_a[j]['href'])
        
#create a txt file where for each row there is a book's url 
with open("url.txt", 'w') as file:
    #write on the url.txt the url of the book
    file.write("\n".join(map(str, url)))

In [2]:
1.2
#open the url.txt and read all the lines
file = open('url.txt', "r")
lines=file.readlines()#pass the lines to a array/list
file.close()
#HERE YOU CHANGE THE DIRECTORY TO SAVE THE HTML IN YOUR OWN COMPUTER, THIS IS MY CHOOSEN PATH FOR MY PC
path="D:/ADM/Homework 3/database"
#THIS FIRST FOR IS TO LOAD THE PAGES
#MY SUGGESTION
#- Mateus DOWNLOAD FROM 1 TO 101
#- Ousainou DOWNLOAD FROM 101 TO 201
#- Camila DOWNLOAD FROM 201 TO 301
#TOTAL OF PAGES ADJUSTED TO THE FOR INDEX - FROM 1 TO 301 - TOTAL 300 PAGES
for page in range(1,301):
    name="page"+str(page)
    pathmaster = os.path.join(path, name)
    os.makedirs(pathmaster)
    #AS EACH PAGE HAS 100 BOOKS, DO NOT CHANGE THIS FOR BECAUSE IT IS RESPONSIBLE TO SEPARETE THE CORRECT NUMBER OF BOOK FOR EACH PAGE
    for line in range (1,101):
        site=lines[line]
        access=requests.get(site, allow_redirects=True)
        book_site = "article_" + str(line) + ".html"
        open(pathmaster+"/"+book_site,'wb').write(access.content)

In [4]:
#open the url.txt and read all the lines
file = open('url.txt', "r")
lines=file.readlines()#pass the lines to a array/list
file.close()
#HERE YOU CHANGE THE DIRECTORY TO SAVE THE HTML IN YOUR OWN COMPUTER, THIS IS MY CHOOSEN PATH FOR MY PC
for i in range(1,301):
    #here is the access of where we are gonna save the tsv files
    page_folder_path="D:/ADM/Homework 3/database/page"+str(i)+"/"
    for j in range(1,101):
        #for each article of the page, first we get the less complex informations as mentioned below
        article_saved='article_'+str(j)+'.html'
        teste=str(page_folder_path+article_saved)
        soup = BeautifulSoup(open(teste, 'r', encoding="utf8"), features='lxml')
        bookTitle = str(soup.find_all('h1', id = "bookTitle")[0].contents[0]).replace('\n', '').strip()
        bookAuthors = str(soup.find_all('a', {"class": "authorName"})[0].contents[0].contents[0])
        ratingValue = soup.find_all('span', itemprop = "ratingValue")[0].contents[0].replace('\n', '')
        ratingCount = soup.find_all('a', href="#other_reviews")[0].contents[2].replace('\n', '').strip().split()[0]
        reviewCount = soup.find_all('a', href="#other_reviews")[1].contents[2].replace('\n', '').strip().split()[0]
        NumberofPages = soup.find_all('span', itemprop="numberOfPages")[0].contents[0]
        Published = soup.find_all('div', {"class": "row"})[1].contents[0].replace('\n', '').strip().split(" "*8)[1]
        aux=str(soup.find_all('link',rel='canonical')).split()
        Url=aux[1].strip('href=').strip('"')
        aux1 = soup.find_all('div', {"class": "infoBoxRowTitle"})
        index = False
        for stri in aux1:
            if 'Series' in stri:
                index = aux1.index(stri)
        bookSeries = ""
        if index:
            bookSeries += str(soup.find_all('div', {"class": "infoBoxRowItem"})[index].contents[1]).strip().split(">")[1][:-3]
        #now we treat PLOT, CHARACTERS AND SETTINGS because we need an different approach to clean the information
        #PLOT
        #first of all, define the full description
        if (len(soup.find_all('div', id="description")[0].contents) <= 3):
            descrip = soup.find_all('div', id="description")[0].contents[1].contents
        else :
            descrip = soup.find_all('div', id="description")[0].contents[3].contents
            #if the description begins with this kind of description, remove it
        if len(soup.find_all('div', id="description")[0].contents) > 3 :
            if len(soup.find_all('div', id="description")[0].contents[3].contents) > 2:
                if "<i>" in str(descrip[0]):
                    descrip = descrip[3:]
            #remove the <br/> tags and add to the list
        Plot = []
        for q in range(len(descrip)):
            if str(descrip[q]) != '<br/>':
                Plot.append(descrip[q])
            # remove the <i> and </i> tags
        for w in range(len(Plot)):
            if '<i>' in str(Plot[w]):
                Plot[w] = re.sub(r'<i>', '', str(Plot[w]))
                Plot[w] = re.sub(r'</i>', '', str(Plot[w]))

            #if the plot begins in bold, remove the <b> tags
        if "<b>" in str(Plot[0]):
            Plot[0] = Plot[0].contents[0]
            
        for u in range(len(Plot)):
            Plot[u] = str(Plot[u])
                
                
        Plot = " ".join(Plot)

        if Plot == [] :
                
            Plot = ""

        #CHARACTERS
        index = False
        #using the same soup created above, we check if there is a setting inside the soup    
        for stri in aux1:
            if 'Characters' in stri:
                index = aux1.index(stri)
        characters = []
        #if character exists, we will find them up to 5 characters.
        if index:
            if int(len(soup.find_all('div', {"class": "infoBoxRowItem"})[index].contents)/2) <= 5 :
                for e in range(int(len(soup.find_all('div', {"class": "infoBoxRowItem"})[index].contents)/2)):
        #using 2i+1 because character is on odd index - name and surname
                    characters.append(str(soup.find_all('div', {"class": "infoBoxRowItem"})[index].contents[2*e+1]).strip().split(">")[1][:-3])
            else:
                for e in range(5):
                    characters.append(str(soup.find_all('div', {"class": "infoBoxRowItem"})[index].contents[2*e+1]).strip().split(">")[1][:-3])
            characters = ", ".join(characters)
        else :
            characters = ""
        setting = []
        index = 0
        for check in aux1:
        #using the same soup created above, we check if there is a setting inside the soup    
            if 'Setting' in check:
                index = aux1.index(check)
        if index > 0:
            for r in range(2):
                if "</s" in str(soup.find_all('div', {"class": "infoBoxRowItem"})[index].contents[2*r+1]) :
                        temp = re.sub(r'</s', '', str(soup.find_all('div', {"class": "infoBoxRowItem"})[index].contents[2*r+1]))
                        setting.append((str(temp).strip().split(">")[1][:-3]).strip())
                else:
                    setting.append(str(soup.find_all('div', {"class": "infoBoxRowItem"})[index].contents[2*r+1]).strip().split(">")[1][:-3])
            setting = " ".join(setting)
        else :
            setting = ""
        #DEFINE IF PLOT IS IN ENGLISH
        if detect(Plot) == 'en':     
            out_path = page_folder_path + "article_" + str(j) + ".tsv"
        with open(out_path, 'wt', encoding="utf8") as out_file:
            tsv_writer = csv.writer(out_file, delimiter='\t')
            tsv_writer.writerow(['bookTitle', 'bookAuthors', 'ratingValue', 'ratingCount', 'reviewCount', 'Plot', 'NumberofPages', 'Published', 'Characters', 'Setting', 'Url'])
            tsv_writer.writerow([bookTitle, bookSeries, bookAuthors, ratingValue, ratingCount, reviewCount, Plot, NumberofPages, Published, characters, setting, Url])